<a href="https://colab.research.google.com/github/akshatjaiin/woman-safety-gem-/blob/main/SOSHandSign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'handsignimages:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F589922%2F1064771%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240829%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240829T115824Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2f3382690e177d8745a4248b3d04a800a6c814788e7c4a711290de91d81e99acabb5fb4249a18a49da4e7af9c4208028969678733e10e2b64d1ca7652959be0043b0e2d48869eddf1cd212462c9afc99c9408d95a332127aff9f1548af3c8bd442efe1d431777e1ca7d5dbfd6003eb45940cbfd93601166f00a293075a08605a17999512c1672cd80adc69516fd83020e00399b38df7c1bad3594465e20e1cbdff23cf78388b2a2824a1b3426e0da99bfa7b37f7e92011bc8ae0db73c423057bdbb3f5a9f358ed1490553733e9589e3ab04cd5308b15bcbe4ccaf0f89fdc6cfae6ea161676712754c60a306436232123ef14f6ee8f12801c1d12a016fedd2af4'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 24265584 bytes downloaded
Downloaded and uncompressed: handsignimages
Data source import complete.


In [21]:
from tensorflow.python.client import device_lib

!export CUDA_VISIBLE_DEVICES=0
print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7649039282204433688
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14626652160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10875359378288645467
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]
Num GPUs Available:  1


In [22]:
import numpy as np
import pandas as pd
import cv2;
import os;
from glob import glob;
from sklearn.model_selection import train_test_split;
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from sklearn.utils import shuffle;
from tensorflow.keras.utils import to_categorical;
from tensorflow.keras.applications import MobileNetV2
import random
from keras.callbacks import EarlyStopping;

In [23]:
ImagesPath = "/kaggle/input/handsignimages/Train"
ImagesType = "*"
ImageSize = (64,64,3)

This is for finding file name for other hand signs :)

In [24]:
import itertools

otherHands = list(itertools.chain(*[glob(os.path.join(i,ImagesType))[:46:2] for i in glob(os.path.join(ImagesPath,"*"), recursive=True) if i[-1]not in ("M","B")]));
random.shuffle(otherHands);
print(len(otherHands))
otherHands = otherHands[:200];
print(len(otherHands))

506
200


In [26]:
dataB = glob(os.path.join(ImagesPath+"/B",ImagesType))[:200];
dataM = glob(os.path.join(ImagesPath+"/M",ImagesType))[:200];

In [27]:
dataReturns = {
    "B":1,"M":2
}
def ImageExtractor(data,size,mode=0,Y=[]):
    tmpX , tmpY = [],[];
    for i in range(len(data)):
        img = cv2.imread(data[i]);
        img_resized = cv2.resize(img, size[0:2]);
        if(mode and Y):
            angle  = random.randint(1,45);
            tmpX.append(np.array(img_resized,dtype=float) / 255.0);
            img_rotate = [np.array(cv2.warpAffine(img_resized,cv2.getRotationMatrix2D((img_resized.shape[1]/2, img_resized.shape[0]/2), angle, 1)
,(img_resized.shape[1], img_resized.shape[0])))/255.0 for i in range(int(360/angle)-1)];
            tmpY.append(Y[i]);
            tmpX.extend(img_rotate);
            tmpY.extend([Y[i]]*(len(img_rotate)));
        else:
            data[i]=np.array(img_resized,dtype=float) / 255.0;

    if(mode and Y): return tmpX,tmpY
    return data

In [28]:
X , Y = [] , [];
X.extend(otherHands);
Y.extend([0]*len(otherHands))
X.extend(dataB);
Y.extend([dataReturns["B"]]*len(dataB))
X.extend(dataM);
Y.extend([dataReturns["M"]]*len(dataM))
len(X)


600

In [29]:
X,Y = ImageExtractor(X,ImageSize[:2],mode=1,Y=Y)
print(len(X),len(Y))
X, Y = shuffle(X, Y, random_state=42)
X = np.array(X);
Y = to_categorical(np.array(Y),num_classes=3);

23757 23757


In [30]:
# Test Data Import
ImagesPathTest = "/kaggle/input/handsignimages/Test"
ImagesTypeTest = "*.jpg"
testY = []
testX = list(itertools.chain(*[glob(os.path.join(i,ImagesType))[:23:2] for i in glob(os.path.join(ImagesPath,"*"), recursive=True) if i[-1]not in ("M","B")]));
testY.extend([0]*len(testX));
testX.extend(glob(os.path.join(ImagesPath+"/B",ImagesType))[:23:2]);
testY.extend([dataReturns["B"]]*(len(testX)-len(testY)));
testX.extend(glob(os.path.join(ImagesPath+"/M",ImagesType))[:23:2]);
testY.extend([dataReturns["M"]]*(len(testX)-len(testY)));
testX = ImageExtractor(testX,ImageSize[:2]);
testX , testY = shuffle(testX,testY,random_state=42);
testX = np.array(testX);
testY = to_categorical(np.array(testY), num_classes=3);
print(testX.shape,testY.shape);

(288, 64, 64, 3) (288, 3)


Model

In [34]:
base_model = MobileNetV2(input_shape=ImageSize, include_top=False, weights='imagenet')
layer = GlobalAveragePooling2D()(base_model.output);
layer = Dense(128, activation='relu')(layer)
layer = Dropout(0.5)(layer)
layers = Dense(3, activation='softmax')(layer)

model = Model(inputs=base_model.input,outputs=layers)
for l in base_model.layers: l.trainable=True;
model.compile(optimizer='adamW', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(
    monitor='val_loss',      # Metric to monitor
    patience=10,             # Number of epochs with no improvement to wait before stopping
    restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored metric
)
model.summary();

<ipython-input-34-6bcb6fadcbca>:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=ImageSize, include_top=False, weights='imagenet')


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 32, 32, 32)     │            864 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 32, 32, 32)     │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 32, 32, 32)     │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 32, 32, 32)     │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 32, 32, 32)     │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 32, 32, 32)     │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 32, 32, 16)     │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 32, 32, 16)     │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 32, 32, 96)     │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 32, 32, 96)     │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 32, 32, 96)     │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 33, 33, 96)     │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 16, 16, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├──────────────────────

 Total params: 2,422,339 (9.24 MB)

 Trainable params: 2,388,227 (9.11 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [35]:
with tf.device('/device:GPU:0'):
    model.fit(X, Y, epochs=50, batch_size=128, validation_data=(testX, testY),callbacks=[early_stopping])

Epoch 1/50
186/186 ━━━━━━━━━━━━━━━━━━━━ 91s 194ms/step - accuracy: 0.9275 - loss: 0.2034 - val_accuracy: 0.0799 - val_loss: 52.7906
Epoch 2/50
186/186 ━━━━━━━━━━━━━━━━━━━━ 30s 49ms/step - accuracy: 0.9951 - loss: 0.0208 - val_accuracy: 0.2882 - val_loss: 47.1369
Epoch 3/50
186/186 ━━━━━━━━━━━━━━━━━━━━ 10s 50ms/step - accuracy: 0.9990 - loss: 0.0039 - val_accuracy: 0.3333 - val_loss: 44.3956
Epoch 4/50
186/186 ━━━━━━━━━━━━━━━━━━━━ 9s 47ms/step - accuracy: 0.9994 - loss: 0.0026 - val_accuracy: 0.4618 - val_loss: 30.5943
Epoch 5/50
186/186 ━━━━━━━━━━━━━━━━━━━━ 10s 49ms/step - accuracy: 1.0000 - loss: 6.2991e-05 - val_accuracy: 0.6146 - val_loss: 13.9246
Epoch 6/50
186/186 ━━━━━━━━━━━━━━━━━━━━ 10s 49ms/step - accuracy: 0.9999 - loss: 2.2009e-04 - val_accuracy: 0.5486 - val_loss: 11.2336
Epoch 7/50
186/186 ━━━━━━━━━━━━━━━━━━━━ 10s 49ms/step - accuracy: 0.9940 - loss: 0.0274 - val_accuracy: 0.3264 - val_loss: 14.7048
Epoch 8/50
186/186 ━━━━━━━━━━━━━━━━━━━━ 9s 47ms/step - accuracy: 0.9982 - l

In [36]:
model.summary();

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 32, 32, 32)     │            864 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 32, 32, 32)     │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 32, 32, 32)     │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 32, 32, 32)     │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 32, 32, 32)     │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 32, 32, 32)     │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 32, 32, 16)     │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 32, 32, 16)     │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 32, 32, 96)     │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 32, 32, 96)     │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 32, 32, 96)     │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 33, 33, 96)     │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 16, 16, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├──────────────────────

 Total params: 7,198,795 (27.46 MB)

 Trainable params: 2,388,227 (9.11 MB)

 Non-trainable params: 34,112 (133.25 KB)

 Optimizer params: 4,776,456 (18.22 MB)

In [37]:
loss , acc = model.evaluate(testX,testY);
print(acc,loss);

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9985 - loss: 0.0070
0.9965277910232544 0.01041952520608902


In [46]:
# !rm SosSign.h5
from  tensorflow.keras.saving import save_model
save_model(model,"Sos_sign_v2.h5")
# model.save("SosSign.h5")

In [47]:
from IPython.display import Image,display
testFiles = ["/kaggle/input/handsignimages/Test/A/112_A.jpg","/kaggle/input/handsignimages/Test/B/932_B.jpg","/kaggle/input/handsignimages/Test/X/834_X.jpg","/kaggle/input/handsignimages/Test/M/230_M.jpg"];
for i in testFiles:
  print(i)
  display(Image(str(i)))

/kaggle/input/handsignimages/Test/A/112_A.jpg


/kaggle/input/handsignimages/Test/B/932_B.jpg


/kaggle/input/handsignimages/Test/X/834_X.jpg


/kaggle/input/handsignimages/Test/M/230_M.jpg


In [48]:
my_test = ImageExtractor(testFiles,ImageSize[:2]);
my_test = np.array(my_test);
print(my_test.shape);

(4, 64, 64, 3)


In [49]:
pred = model.predict(my_test);
for i in range(len(pred)):
  print("res for ",i+1," num pic")
  print(np.argmax(pred[i]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
res for  1  num pic
0
res for  2  num pic
1
res for  3  num pic
0
res for  4  num pic
2


In [73]:
print([i for i in Y if i[2]]);

[array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0